# Libraries

In [1]:
from pathlib import Path
import os
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

from skimage.feature import local_binary_pattern

In [2]:
subfolder_path = Path.cwd()
notebooks_path = subfolder_path.parent
repo_path = notebooks_path.parent
os.chdir(str(subfolder_path))
#print current working directory
print(f'Current folder is: {os.getcwd()}\n')
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent.parent)) if sys.path[0] != str(thispath.parent.parent) else None

#Import paths and patients classes
from notebooks.info import path_label, patient


Current folder is: /home/ricardino/Documents/MAIA/tercer_semestre/CAD/Projecte/Machine_Learning/notebooks/features



# Functions

In [3]:
#print 3 channel image given its numpy array
def show_image_3channels(image):
    """Print 3 channel image given its numpy array
    """
    plt.figure(figsize=(10,10))
    plt.imshow(image)
    plt.show()
    
#functio to save file as pickle
def save_as_pickle(file, filename):
    """Save file as pickle
    """
    with open(filename, 'wb') as f:
        pickle.dump(file, f, pickle.HIGHEST_PROTOCOL)
def load_pickle(filename):
    """Load pickle file
    """
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [4]:
def get_LBP_3channels(im_chs, neighbors=12, radius=2, method='uniform'):
    """Get LBP features from 3 channels

    Args:
        im_chs (3D array): 3 channels image
        neighbors (int, optional): number of neighbors. Defaults to 12.
        radius (int, optional): radious to extract. Defaults to 2.
        method (str, optional): normally uniform is used but others are available. Defaults to 'uniform'.

    Returns:
        array: size 3*number of LBP features
    """
    lbp = local_binary_pattern(im_chs[:,:,0], P=neighbors, R=radius, method=method)
    lbp_count_1, _ = np.histogram(lbp, density = True, bins=10, range = (0, int(lbp.max())+1))
    lbp = local_binary_pattern(im_chs[:,:,1], P=neighbors, R=radius, method=method)
    lbp_count_2, _ = np.histogram(lbp, density = True, bins=10, range = (0, int(lbp.max())+1))
    lbp = local_binary_pattern(im_chs[:,:,2], P=neighbors, R=radius, method=method)
    lbp_count_3, _ = np.histogram(lbp, density = True, bins=10, range = (0, int(lbp.max())+1))
    
    return np.concatenate((lbp_count_1, lbp_count_2, lbp_count_3))


# IMP

In [8]:
#Read meta information
classif = 'binary'
NH_status = True #Hair removal yes or no
fname = 'lbp_hsv'

for set_name in ['test']:

    #Get information for this case
    info = path_label(classif = classif, set_name=set_name)

    lbp_matrix = np.zeros((info.len, 30)) #define lbp matrix
    for i in tqdm(range(info.len)):
        pacient = patient(info=info, num=i, NH = NH_status)
        lbp_matrix[i] = get_LBP_3channels(pacient.HSV_im(), neighbors=12, radius=2, method='uniform')

    #save matrix as pickle
    save_as_pickle(lbp_matrix, str(repo_path) + f'/data/features/texture/{classif}_{set_name}_{fname}_fv.p')

100%|██████████| 6340/6340 [22:53<00:00,  4.62it/s]


# Concatenate all LBP features

In [9]:
classif = 'binary'
set_name = 'test'

#Concatenate all 3 channels LBP features
lbp_rgb = load_pickle(str(repo_path) + f'/data/features/texture/{classif}_{set_name}_lbp_rgb_fv.p')
lbp_hsv = load_pickle(str(repo_path) + f'/data/features/texture/{classif}_{set_name}_lbp_hsv_fv.p')
lbp_ycrcb = load_pickle(str(repo_path) + f'/data/features/texture/{classif}_{set_name}_lbp_ycrcb_fv.p')
lbp_features = np.concatenate((lbp_rgb, lbp_hsv, lbp_ycrcb), axis=1)
save_as_pickle(lbp_features, str(repo_path) + f'/data/features/texture/{classif}_{set_name}_lbp_NH_fv.p')